In [2]:
import sys 

# Install all dependencies
#!{sys.executable} -m pip install numpy scipy matplotlib pyvis 

# Change path as the folder adress you have put the library in 
path = "C:\\Users\\Paul Valcke\\Documents\\GitHub\\GEMMES" 
sys.path.insert(0, path)
import pygemmes as pgm
import plotly.graph_objects as go
import numpy as np
import pandas as pd


pyIDEE

* Version 0.9
* Last update 2023/01/03
* Developped at the environmental justice program https://environmentaljustice.georgetown.edu/#
* Contact : Paul Valcke pv229@georgetown.edu

Welcome in PyIDEE, a modular library to prototype and study dynamical systems !
This library is oriented toward generation of macroeconomic complexity models

If you find bugs, want some new extensions, or help us improve the library, please create a new issue on github
If this is the first time you open this library, please look at the tutorial file in doc/tutorial.py or better, and execute it line by line.

The ipythonNotebook is at : C:\Users\Paul Valcke\Documents\GitHub\GEMMES\pygemmes\doc\TUTORIALS\TUTORIAL.ipynb
If you want to customize pyIDEE (advancer users) like removing this message, edit : C:\Users\Paul Valcke\Documents\GitHub\GEMMES\pygemmes\_config.py



In [3]:

hub=pgm.Hub('ECHIMES',preset='SimpleTri',verb=False)
hub.run()


Exception: The model you asked, ECHIMES, cannot be found. Found models are 
...Maybe you mispelled it ?

In [ ]:
#hub.plot(filters_key=['pi'])

In [3]:

listX=['Minvest','Minter']
nx=0
region=0
ntindex=0
R=hub.get_dparam()

c = [ 'rgba(255,0,255, 0.8)' , 'rgba(0,255,255, 0.8)' , 'rgba(255,255,0, 0.8)','rgba(127,255,127, 0.8)' ]

### INITIAL LISTS TO FILL
source=[] # Where the flux starts
target=[] # Where the flux ends
value =[] # Flux intensity
types =[]
colors=[]
label =[] 

### MATRICES
for ii,X in enumerate(listX):
    d0=R[X]
    values = d0['value'][ntindex,nx,region,:,:].reshape(-1)
    names = R[d0['size'][0]]['list']
    sectintdex=np.arange(len(names))

    XX,YY=np.meshgrid(sectintdex+1,sectintdex+1)
    XX=XX.astype(int).reshape(-1)-1
    YY=YY.astype(int).reshape(-1)-1
    Ntot=len(XX)
    
    source.extend(YY)
    target.extend(XX)
    types.extend([X for i in range(len(XX))])
    value.extend(values)
    colors.extend([c[ii] for j in range(len(XX))])
    label = names


### ADDING VECTORS
Vecid = 'C'
stype = 'Consumption'
slabel = 'Household'
V=R[Vecid]['value'][ntindex,nx,region,:,0]        # Values we add 
#if slabel not in label:
label.append(slabel)
target.extend([np.amax(target+source)+1]*len(V))
source.extend(np.arange(0,len(V)))
value.extend(V)
types.extend([stype for i in range(len(V))])
colors.extend([c[ii+1] for ii in range(len(V))])

tempdict={          #'label':label,
                   'target':target,
                   'source':source,
                   'value':value,
                   'types':types,
                   'colors':colors}
df = pd.DataFrame(tempdict)


link = dict(source = np.array(source).reshape(-1), 
                             target = np.array(target).reshape(-1), 
                             value = np.array(value).reshape(-1),
                             color=colors)
node = dict(  label = label,
                             pad=50, 
                             thickness=5)
data = go.Sankey(link = link , 
                 node= node)
# plot
figPhy = go.Figure(data)
figPhy.update_layout(
    hovermode = 'x',
    title=f"Physical exchanges between sectors, t={R['time']['value'][ntindex,0,0,0,0]:.2f}",
    font=dict(size = 10, color = 'white'),
    paper_bgcolor='#5B5958'
)
figPhy.show()

In [18]:
#### MONETARY FLUXES 
Matrices=['MtransactI','MtransactY']
Vectors =['wL','pC','rD']
nx=0
region=0
ntindex=-1
R=hub.get_dparam()

c = [ 'rgba(255,  0,255, 0.8)' , 
      'rgba(  0,255,255, 0.8)' ,
      'rgba(255,255,  0, 0.8)' ,
      'rgba(127,255,127, 0.8)' ,
      'rgba(0,0,0, 0.8)' ,
      'rgba(127,255,255, 0.8)' ,
      'rgba(255,127,255, 0.8)']

### INITIAL LISTS TO FILL
TD={               'label' :[], # names of target/sources
                   'target':[], # Where the flux ends
                   'source':[], # Where the flux starts
                   'value' :[], # Flux intensity
                   'types' :[], # Flux category 
                   'colors':[]}

def Add_Matrix(X):
    d0=R[X]
    values = d0['value'][ntindex,nx,region,:,:].reshape(-1)
    names = R[d0['size'][0]]['list']
    sectintdex=np.arange(len(names))

    XX,YY=np.meshgrid(sectintdex+1,sectintdex+1)
    XX=XX.astype(int).reshape(-1)-1
    YY=YY.astype(int).reshape(-1)-1
    Ntot=len(XX)
    
    TD['source'].extend(YY)
    TD['target'].extend(XX)
    TD['types'].extend([X for i in range(len(XX))])
    TD['value'].extend(values)
    TD['colors'].extend([len(set(TD['types']))-1 for j in range(len(XX))])
    TD['label'] = names
    return TD

def Add_Vector(TD,Vecid,stype,slabel,switch):
    V=R[Vecid]['value'][ntindex,nx,region,:,0]        # Values we add 
    if slabel not in TD['label']:
        TD['label'].append(slabel)
        plus= 1
    else :
        plus=0
    V1 = np.arange(0,len(V))
    V2 = [np.amax(TD['target']+TD['source'])+plus]*len(V)
    TD['target'].extend(V1 if switch else V2) 
    TD['source'].extend(V2 if switch else V1)
    TD['value'].extend(V)
    TD['types'].extend([stype for i in range(len(V))])
    TD['colors'].extend([len(set(TD['colors'])) for ii in range(len(V))])
    return TD

def Add_scalar(TD,name,source,target,type,newcolor=False):
    print(source,TD['label'])
    if source in TD['label']:
        sourceindex = TD['label'].index(source)
    else : 
        print(source,'adding')
        sourceindex = len(TD['label'])
        TD['label'].append(source)

    if target in TD['label']:
        targetindex = TD['label'].index(target)
    else : 
        targetindex = len(TD['label'])
        TD['label'].append(target)

    TD['source'].append(sourceindex)
    TD['target'].append(targetindex)
    TD['value'].append(R[name]['value'][ntindex,nx,region,0,0])
    TD['types'].append(type)
    TD['colors'].append(TD['colors'][-1]+1 if newcolor else TD['colors'][-1])
    return TD    


### MATRICES
for ii,X in enumerate(Matrices):
    Add_Matrix(X)



### ADDING VECTORS
Vecid = 'pC'
stype = 'Consumption'
slabel = 'Household'
switch = True
TD = Add_Vector(TD,Vecid,stype,slabel,switch)

Vecid = 'wL'
stype = 'Wages'
slabel = 'Household'
TD = Add_Vector(TD,Vecid,stype,slabel,False)

Vecid = 'rD'
stype = 'Interests'
slabel = 'Banks'
TD = Add_Vector(TD,Vecid,stype,slabel,False)

TD = Add_scalar(TD,'rDh','Household','Banks','Interests',False)


TD['colors']= [c[i] for i in TD['colors']]
for i,v in enumerate(TD['value']):
    if v<0:
        TD['target'][i],TD['source'][i] = TD['source'][i],TD['target'][i]
        TD['value'][i]*=-1


df = pd.DataFrame({k:v for k,v in TD.items() if k!='label'})


link = dict(source = np.array(TD['source']).reshape(-1), 
                             target = np.array(TD['target']).reshape(-1), 
                             value = np.array(TD['value']).reshape(-1),
                             color=TD['colors'])
node = dict(  label = TD['label'],
                             pad=50, 
                             thickness=5)
data = go.Sankey(link = link , 
                 node= node)
# plot
figMoney = go.Figure(data)
figMoney.update_layout(
    hovermode = 'x',
    title=f"Monetary exchanges between sectors, t={R['time']['value'][ntindex,0,0,0,0]:.2f}",
    font=dict(size = 10, color = 'white'),
    paper_bgcolor='#5B5958'
)
figMoney.show()

Household ['Consumption', 'Intermediate', 'Capital', 'Household', 'Banks']


In [61]:
def Sankey(hub,t=0,idx=0,Region=0):
    '''Physical and monetary Sankey diagrams'''

    if hub.dmodel['name'] not in ['ECHIMES','CHIMES0']:
        print('CAREFUL IT CAN ONLY WORKS ON ECHIMES RELATED MODELS')

    c = ['rgba(255,  0,255, 0.8)', # Colors 
        'rgba(  0,255,255, 0.8)' ,
        'rgba(255,255,  0, 0.8)' ,
        'rgba(127,255,127, 0.8)' ,
        'rgba(0,0,0, 0.8)'       ,
        'rgba(127,255,255, 0.8)' ,
        'rgba(255,127,255, 0.8)']

    R=hub.get_dparam()
    ##################### TRANSLATING INPUT #################
    # idx input
    if type(idx)==int: pass 
    elif type(idx)==str:
        try : idx=hub.dparam['nx']['list'].index(idx)
        except BaseException:
            liste=hub.dparam['nx']['list']
            raise Exception(f'the parrallel system cannot be found !\n you gave {idx} in {liste}')
    else: raise Exception(f'the parrallel index cannot be understood ! you gave {idx}')

    # Region input 
    if type(Region)==int: pass 
    elif type(Region)==str:
        try : Region=hub.dparam['nr']['list'].index(Region)
        except BaseException:
            liste=hub.dparam['nr']['list']
            raise Exception(f'the region system cannot be found !\n you gave {idx} in {liste}')
    else: raise Exception(f'the region index cannot be understood ! you gave {idx}')

    # time input
    time = R['time']['value'][:,idx,Region,0,0]
    if t: ntindex=np.argmin(np.abs(time-t))
    else : ntindex=0
    ###############################################################
    ### LOCAL FUNCTIONS ###########################################
    def Add_Matrix(X,TDi):
        R=hub.get_dparam()
        d0=R[X]
        values = d0['value'][ntindex,idx,Region,:,:].reshape(-1)
        names = R[d0['size'][0]]['list']
        sectintdex=np.arange(len(names))

        XX,YY=np.meshgrid(sectintdex+1,sectintdex+1)
        XX=XX.astype(int).reshape(-1)-1
        YY=YY.astype(int).reshape(-1)-1
    
        TDi['source'].extend(YY)
        TDi['target'].extend(XX)
        TDi['types'].extend([X for i in range(len(XX))])
        TDi['value'].extend(values)
        TDi['colors'].extend([len(set(TDi['types']))-1 for j in range(len(XX))])
        TDi['label'] = names

        #print('MATRIX',X)
        #print(values)
        #for k,v in TDi.items(): print(k,len(v),v)
        
        return TDi
        
    def Add_Vector(TD,Vecid,stype,slabel,switch):
        V=R[Vecid]['value'][ntindex,idx,Region,:,0]        # Values we add 
        if slabel not in TD['label']:
            TD['label'].append(slabel)
            plus= 1
        else :
            plus=0
        V1 = np.arange(0,len(V))
        V2 = [np.amax(TD['target']+TD['source'])+plus]*len(V)
        TD['target'].extend(V1 if switch else V2) 
        TD['source'].extend(V2 if switch else V1)
        TD['value'].extend(V)
        TD['types'].extend([stype for i in range(len(V))])
        TD['colors'].extend([len(set(TD['colors'])) for ii in range(len(V))])
        return TD

    def Add_scalar(TD,name,source,target,type,newcolor=False):
        if source in TD['label']:
            sourceindex = TD['label'].index(source)
        else : 
            print(source,'adding')
            sourceindex = len(TD['label'])
            TD['label'].append(source)

        if target in TD['label']:
            targetindex = TD['label'].index(target)
        else : 
            targetindex = len(TD['label'])
            TD['label'].append(target)

        TD['source'].append(sourceindex)
        TD['target'].append(targetindex)
        TD['value'].append(R[name]['value'][ntindex,idx,Region,0,0])
        TD['types'].append(type)
        TD['colors'].append(TD['colors'][-1]+1 if newcolor else TD['colors'][-1])
        return TD    

    for _ in range(1):
        ###############################################################
        ### PHYSICAL FLUXES ###########################################
        TDm={           'label' :[], # names of target/sources
                        'target':[], # Where the flux ends
                        'source':[], # Where the flux starts
                        'value' :[], # Flux intensity
                        'types' :[], # Flux category 
                        'colors':[]}

        ### MATRICES
        for X in ['Minvest','Minter']: TDm=Add_Matrix(X,TDm)

        ### ADDING VECTORS
        TDm = Add_Vector(TDm,'C','Consumption','Household',False)
        
        ### INVERSE VECTORS
        for i,v in enumerate(TDm['value']):
            if v<0:
                TDm['target'][i],TDm['source'][i] = TDm['source'][i],TDm['target'][i]
                TDm['value'][i]*=-1

        TDm['colors']= [c[i] for i in TDm['colors']]

        data = go.Sankey(link = dict(source = np.array(TDm['source']).reshape(-1), 
                                    target = np.array(TDm['target']).reshape(-1), 
                                    value = np.array(TDm['value']).reshape(-1),
                                    color=TDm['colors']), 
                        node = dict(label = TDm['label'],
                                    pad=50, 
                                    thickness=5))

        figPhy = go.Figure(data)
        figPhy.update_layout(
            hovermode = 'x',
            title=f"Physical exchanges between sectors, t={R['time']['value'][ntindex,0,0,0,0]:.2f}",
            font=dict(size = 10, color = 'white'),
            paper_bgcolor='#5B5958'
        )
        figPhy.show()

    ###############################################################
    #### MONETARY FLUXES ##########################################
    for _ in range(1):
        Matrices=['MtransactI','MtransactY']

        ### INITIAL LISTS TO FILL
        TD={            'label' :[], # names of target/sources
                        'target':[], # Where the flux ends
                        'source':[], # Where the flux starts
                        'value' :[], # Flux intensity
                        'types' :[], # Flux category 
                        'colors':[]}

        ### MATRICES
        for X in Matrices: TD=Add_Matrix(X,TD)

        ### ADDING VECTORS
        TD = Add_Vector(TD,'pC','Consumption','Household',True)
        TD = Add_Vector(TD,'wL','Wages','Household',False)
        TD = Add_Vector(TD,'rD','Interests','Banks',False)

        ### Adding scalar
        TD = Add_scalar(TD,'rDh','Household','Banks','Interests',False)
        TD['colors']= [c[i] for i in TD['colors']]

        for i,v in enumerate(TD['value']):
            if v<0:
                TD['target'][i],TD['source'][i] = TD['source'][i],TD['target'][i]
                TD['value'][i]*=-1

        data = go.Sankey(link = dict(source = np.array(TD['source']).reshape(-1), 
                                    target = np.array(TD['target']).reshape(-1), 
                                    value = np.array(TD['value']).reshape(-1),
                                    color=TD['colors']), 
                        node= dict( label = TD['label'],
                                    pad=50, 
                                    thickness=5))

        # plot
        figMoney = go.Figure(data)
        figMoney.update_layout(
            hovermode = 'x',
            title=f"Monetary exchanges between sectors, t={R['time']['value'][ntindex,0,0,0,0]:.2f}",
            font=dict(size = 10, color = 'white'),
            paper_bgcolor='#5B5958'
        )
        figMoney.show()

In [62]:
Sankey(hub,t=50,idx=0,Region=0)

IntSlider(value=3, description='Sankey time index :', max=500)

226


['Accordion', 'AppLayout', 'Audio', 'BoundedFloatText', 'BoundedIntText', 'Box', 'Button', 'ButtonStyle', 'CallbackDispatcher', 'Checkbox', 'Color', 'ColorPicker', 'ColorsInput', 'Combobox', 'Controller', 'CoreWidget', 'DOMWidget', 'DatePicker', 'Datetime', 'DatetimePicker', 'Dropdown', 'FileUpload', 'FloatLogSlider', 'FloatProgress', 'FloatRangeSlider', 'FloatSlider', 'FloatText', 'FloatsInput', 'GridBox', 'GridspecLayout', 'HBox', 'HTML', 'HTMLMath', 'Image', 'IntProgress', 'IntRangeSlider', 'IntSlider', 'IntText', 'IntsInput', 'Label', 'Layout', 'NaiveDatetimePicker', 'NumberFormat', 'Output', 'Password', 'Play', 'RadioButtons', 'Select', 'SelectMultiple', 'SelectionRangeSlider', 'SelectionSlider', 'SliderStyle', 'Stack', 'Style', 'Tab', 'TagsInput', 'Text', 'Textarea', 'TimePicker', 'ToggleButton', 'ToggleButtons', 'ToggleButtonsStyle', 'TwoByTwoLayout', 'TypedTuple', 'VBox', 'Valid', 'ValueWidget', 'Video', 'Widget', '__builtins__', '__cached__', '__doc__', '__file__', '__jupyter_